# **Gesture-Recognition System For Drones**

## **Goal:**
Efficiently using a machine learning model and Mediapipe for hand-landmarking to create a system that can reliably predict hand gestures that pass on movement data to the drone.

## **The gestures:**
We will be using a total of 8 gestures to make the tello run
Gestures used are:
<div style="display: inline-block; margin-right: 10%; max-width: 200px; float: right;">
<img
  src="https://s-cdn.ryzerobotics.com/stormsend/uploads/13433930-d1e1-0135-d3c1-12530322f90d/guava-%E7%99%BD-pc-160_154_2x.png"
  alt="Drone Image"
  title="Fig.1 Drone"
  style="border-radius:5%;"><p align="center">Fig.1 Drone</p>
</div>

1. ```Up     - Point Upwards                         (2)```
2. ```Down   - Point Downwards                       (2)```
3. ```Left   - Point to Left                         (2)```
4. ```Right  - Point to Right                        (2)```
5. ```Front  - Flatten hand and Point Forward        (2)```
6. ```Back   - Thumb and Pinky Finger out            (2)```
7. ```Land   - Okay sign                             (2)```
8. ```Flip   - Yo! sign                              (4)```


#### This is  a **four-part project** and the dataset of this is available on kaggle.
The github repository of the project is:<br>
[https://github.com/RumbleJack56/HandGestureRecognition-P](https://github.com/RumbleJack56/HandGestureRecognition-P)


## **Part 1: Data-Collection**

We collect data using ```opencv``` library and use ```cv2.VideoCapture()``` for accessing camera to take images of different gestures as training examples.

* We check for webcams
* Then select the webcam
* and then we click picture every button press (s) and save it in dataset folder